# MeCabの紹介
本紹介は，ツールの歴史的背景や詳細な技術的・構成的情報は簡素化し，ツール利用者向けのチュートリアルを指向しています．ご了承ください．

## MeCabとは
MeCabは，日本語文を形態素と呼ばれる単位の語に分割するツール（形態素解析エンジン）です．
形態素とは，最小の意味を持つ単位であり，「が」や「に」といった助詞も導けます．
MeCabでは，日本語文を単に形態素に分割するだけの「分かち書き」もできますが，品詞なども判別することもできます．

このような形態素解析は，旧来より文書間の類似度計算や，文書検索の技術の前処理として利用されてきました．とはいえ，AI技術により自然言語処理技術が発展してきた今でも**AI技術に日本語文章を学習させるデータを作る**ために形態素解析は必須です．

公式版のMeCabはC++で実装され，2013年に開発がストップしています．一方では，OS (Operating System)や実装言語等の実行環境の進化による不具合を解消するメンテナンスや，MeCabをPythonやCythonから呼び出せるようにしたラッパー実装などが有志により行なわれています．

## fugashiによるMeCab入門

ここでは，導入が容易であり，処理の高速性を考慮して，MeCabをCythonから呼び出せる**fugashi**を紹介します．また，Mecabでは，日本語文章を形態素に分割するのですが，そのためには日本語の形態素とは具体的に何かを知るために**辞書**が必要です．ここでは，良く知られている辞書の一つであるUniDicを利用します．他にも，UniDicの軽量版であるUniDic-liteや，旧来から利用されているIPADicやJUMANDicがあります．

In [ ]:
# UniDic辞書を用いるfugashiのインストール
!pip install fugashi[unidic]
# 辞書(500MB程度)のダウンロード
!python -m unidic download

fugashiでは，Taggerクラスを通じてMeCabの機能を実行します．Tagger(arg)のargには，MeCabへの実行引数を与えることになります．ここでの"-Owakati"は分かち書きを得るための実行引数であり，解析結果の出力が分かち書き（品詞等は除かれる）が得られます．実行引数の一覧は公式的には https://taku910.github.io/mecab/mecab.html にて参照できますが，Webで検索すれば，よりわかりやすい解説サイトが見つかると思います．Taggerクラスのparseメソッドを呼び出して，解析結果となる文字列を得られます．

In [ ]:
from fugashi import Tagger

tagger = Tagger("-Owakati") #-OwakatiはMecabへの実行引数であり，fugashi APIを通じてMecabに渡して実行する．
text = "信州大学工学部は長野市にあります。"
print(tagger.parse(text)) # 分かち書きした結果のみを出力する．

Taggerに対して引数を与えない場合は，形態素ごとに品詞等の情報を得ることができます．出力内容として何が出力されているかの説明は https://clrd.ninjal.ac.jp/unidic/faq.html に任せますが，比較的わかりやすいところで品詞や語彙素（見出し語），仮名等の情報が含まれます．

In [ ]:
from fugashi import Tagger

tagger = Tagger() #実行引数未指定の場合
text = "信州大学工学部は長野市にあります。" #解析対象の文例
print(tagger.parse(text)) # 3行目で何も指定しない場合は，品詞等の情報も出力する

上記でまとめて出力された品詞や語彙素等の情報ですが，分けて得ることもできます．tagger(text)（※textは解析対象の文章）を呼び出すと，品詞等情報を細かく分割して得られるデータが得られます．たとえば品詞分類はpos（やfeature.pos1, feature.pos2, feature.pos3, feature.pos4）で得られますし，語彙素はfeature.lemmaで得られます．

In [ ]:
from fugashi import Tagger

tagger = Tagger()
text = "信州大学工学部は長野市にあります。" #解析対象の文例
for word in tagger(text) :
  print(word.surface, #表層形
        word.feature.pos1, #品詞大分類
        word.feature.pos2, #品詞中分類
        word.feature.pos3, #品詞小分類
        word.feature.pos4, #品詞細分類
        word.feature.cType, #活用型
        word.feature.cForm, #活用形
        word.feature.lForm, #語彙素読み
        word.feature.lemma, #語彙素
        word.feature.orth, #書字形出現形
        word.feature.pron, #発音形出現形
        word.feature.orthBase, #書字形基本形
        word.feature.pronBase, #発音形基本形
        word.feature.goshu, #語種
        word.feature.iType, #語頭変化化型
        word.feature.iForm, #語頭変化形
        word.feature.fType, #語末変化化型
        word.feature.fForm, #語末変化形
        word.feature.iConType, #語頭変化結合型
        word.feature.fConType, #語末変化結合型
        word.feature.type, #
        word.feature.kana, #仮名形出現形
        word.feature.kanaBase, #仮名形基本形
        word.feature.form, #語形出現形
        word.feature.formBase, #語形基本形
        word.feature.aType, #アクセント型
        word.feature.aConType, #アクセント結合型
        word.feature.aModType, #アクセント修飾型
        word.feature.lid, #語彙表ID
        word.feature.lemma_id, #語彙素ID
        sep=' ')

情報を絞り出力する場合は，たとえば以下のようになります．

In [ ]:
from fugashi import Tagger

tagger = Tagger()
text = "信州大学工学部は長野市にあります。" #解析対象の文例
for word in tagger(text) :
  print(word.surface, #表層形
        word.feature.pos1, #品詞大分類
        word.feature.lemma, #語彙素
        sep=' ')

## MeCabの応用例
ここからは形態素解析の応用例として，解析で得られた形態素をカウントしてみます．表層形をカウントしてしまうと，同義の単語にも係わらず活用形の違いから別の単語としてカウントされてしまうことになるため，一般に基本形でカウントするような工夫をします．このような調整を**前処理**と呼ぶことがあり，他にも単体では意味付けが難しい頻出語（例：「は」や「が」といった助詞）を除くようなストップワード処理があります．このような処理は，たとえば文書類似度を計測する等の処理の前処理として旧来から実施されているものになります．

In [ ]:
# 長文テキストの解析をデモする文章例
!wget 'https://www.aozora.gr.jp/cards/000148/files/773_14560.html' -O 'bunko.html' #こころ（青空文庫）

In [ ]:
from bs4 import BeautifulSoup #マークアップ言語による記述からデータを抽出するためのライブラリ
from collections import Counter
from fugashi import Tagger

#単なるtxtファイルに書かれた文章を扱う場合はBeautifulSoupは不要
#そのtxtファイルをopenして文字列を取り出すのみで十分
soup = BeautifulSoup(open("bunko.html", encoding="shift_jis"))
text = soup.find("div", "main_text").text #main_text classのdivタグに本文がある．

tagger = Tagger()
base_list = []
for word in tagger(text) :
  base_list.append(word.feature.orthBase)

#形態素の書字形基本形を収集したリストに対しカウントし辞書化する
count = Counter(base_list)
#カウント数を降順として辞書をソートする
sorted_count = sorted(count.items(), key = lambda word:word[1], reverse=True)
print(dict(sorted_count))


上記コードの出力結果に見られるように，頻出語の上位として``{'の': 5982, 'た': 5523, '。': 4647, 'は': 4181, 'て': 3856, 'に': 3699, '、': 3610, 'を': 3223, 'だ': 3175, 'と': 2698,``のように助詞や句読点が目立ちます．たとえば，当該文書の特徴を捉えようとしたときに，これらの語が多いことは他文書でも同様の可能性があり，文書の特徴を捉えることは困難です．そこで品詞による語のフィルタリングやストップワード処理を実行します．

In [ ]:
#ストップワードとして公開されているテキストを取得
!wget 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt' -O 'stopword.txt'

In [ ]:
from bs4 import BeautifulSoup #マークアップ言語による記述からデータを抽出するためのライブラリ
from collections import Counter
from fugashi import Tagger

#単なるtxtファイルに書かれた文章を扱う場合はBeautifulSoupは不要です．
#そのtxtファイルをopenして文字列を取り出すのみで十分でしょう．
soup = BeautifulSoup(open("bunko.html", encoding="shift_jis"))
text = soup.find("div", "main_text").text #main_text classのdivタグに本文がある．
#ストップワードの読み込み，改行区切りでの単語分割
stopwords = open("stopword.txt", 'r').read().split('\n')
#カウントする形態素の品詞指定
allowed_pos = ["名詞"]

tagger = Tagger()
base_list = []
for word in tagger(text) :
  if word.feature.orthBase not in stopwords and word.feature.pos1 in allowed_pos:
    base_list.append(word.feature.orthBase)

#形態素の書字形基本形を収集したリストに対しカウントし辞書化する
count = Counter(base_list)
#カウント数を降順として辞書をソートする
sorted_count = sorted(count.items(), key = lambda word:word[1], reverse=True)
print(dict(sorted_count))



名詞による語のフィルタリング，ならびにストップワード処理により，実行結果は```{'先生': 597, None: 437, '奥': 401, '父': 296, '母': 184, '嬢': 168, '顔': 133, '言葉': 124, '二人': 115, '眼': 111, '心': 106```となり，文書の特徴的な語が見えるようになってきました．ここで```None```は文書の抽出時が不適切に行われた結果によるノイズでしょう．また，動詞も取り上げて，より顕著な特徴を捉えようとすることもあります．

このような形態素解析の発展的な応用例として，TF-IDF（Term Frequency / Inverse Document Frequency）を計算して文書を特徴づける語として語の重要度を計算することや，LDA (Latent Dirichlet Allocation) 等のトピックモデリング技術により文書を特徴づける語群を検出することがあります　(cf. http://www.mi.u-tokyo.ac.jp/pdf/3-7_language.pdf )．ただし，このような方法は，AI技術による自然言語処理が台頭してきた現代では古典的な手法として紹介されることがあります．

## 演習
1. 手持ちの文書に対し，形態素解析および形態素のカウントを行ない，その結果を示しなさい．
1. 解析結果を表やグラフにより可視化し，どのような可視化が有効かを考察しなさい．

### 参考文献
1. @taku910, MeCab: Yet Another Part-of-Speech and Morphological Analyzer, https://taku910.github.io/mecab/
1. @polm, fugashi, https://github.com/polm/fugashi
1. PyPI, fugashi 1.3.0, https://pypi.org/project/fugashi/
1. @shogo82148, MeCab @shogo82148 flavored: Yet Another Part-of-Speech and Morphological Analyzer, https://shogo82148.github.io/mecab/
1. @SamuraiT, mecab-python3, https://github.com/SamuraiT/mecab-python3
1. Stefan Behnel, et al., Cython C-Extensions for Python, https://cython.org/
1. PyPi, unidic 1.1.0, https://pypi.org/project/unidic/
1. 国立国語研究所, UniDic, https://clrd.ninjal.ac.jp/unidic/download.html
1. PyPi, unidic-lite 1.0.8, https://pypi.org/project/unidic-lite/
1. PyPi, ipadic 1.0.0, https://pypi.org/project/ipadic/
1. PyPi, jumandic 1.0.0, https://pypi.org/project/jumandic/
1. 青空文庫, 青空文庫, https://www.aozora.gr.jp/
1. 東京大学 数理・情報教育研究センター, 3-7 ⾔語・知識, http://www.mi.u-tokyo.ac.jp/pdf/3-7_language.pdf

